In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import poisson
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score, classification_report
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
xgb.set_config(verbosity=0)
from sklearn.feature_selection import RFECV
import random
from sklearn.metrics import mean_squared_error
import sklearn.metrics as sm
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_absolute_error

In [5]:
# Drop first column of dataframe
def drop_first_col(df):
    return df.iloc[: , 1:]

train = pd.read_csv("dataset\\train.csv")
test = pd.read_csv("dataset\\test.csv")

train = drop_first_col(train)
test = drop_first_col(test)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset\\train.csv'

## Test on Autoregressive AR-X with XGBoost

https://cprosenjit.medium.com/multivariate-time-series-forecasting-using-xgboost-1728762a9eeb

In [2]:
import pandas as pd
from dateutil import relativedelta
from statsmodels.tsa.ar_model import AutoReg
# forecastWithAR: method to apply AutoRegression UV forecasting.
# dataset: input time series dataset in Pandas DataFrame format
# group_column: 'group by' attribute to be used for Pandas Func API
# forecast_max_date: end date of the forecasting horizon

def forecastWithAR(dataset, group_column, forecast_max_date):
    group_key = dataset[group_column].iloc[0]

    driver_max_date = dataset.Date.max()
    diff = relativedelta\
            .relativedelta(\
                           forecast_max_date, \
                           driver_max_date)
    forecastingPeriod = diff.months + diff.years * 12

    model_ag = AutoReg(endog = dataset["Value"], \
                     lags = 8, \
                     trend='c', \
                     seasonal = False, \
                     exog = None, \
                     hold_back = None, \
                     period = None, \
                     missing = 'none')
    fit_ag = model_ag.fit()
    predictStart = len(dataset)
    predictEnd = predictStart + \
                forecastingPeriod-1
    predictions = fit_ag.predict(\
                   start=predictStart, \
                   end=predictEnd, \
                   dynamic=False)

    # Assign column header
    predictions.name = "Value"

    # Convert Series to DataFrame
    pred_perf = predictions.to_frame()

    # Assign a column for predicted months
    prediction_start_month = \
    (dataset["Date"].iloc[-1] + \
     pd.DateOffset(months=1))\
    .strftime('%Y-%m-%d')

    pred_date_range = pd.date_range\
      (prediction_start_month, \
       periods=forecastingPeriod, \
       freq='MS')

    pred_perf[group_column] = group_key
    pred_perf['Date'] = pred_date_range

    # return prediction DataFrame
    return pred_perf[[group_column, "Date", "Value"]]

In [3]:
import numpy as np
from datetime import date
from dateutil import relativedelta
# Fetch the schema
schema = actualsDF.select("Driver", "Date", "Value").schema
# The column used for grouping the resultset
group_column = "Driver"
# Total prediction period i.e till 01-Oct-2022
forecast_max_date = date(2022,10,1)
# Python Function
def udf_uv_forecast (dataset:pd.DataFrame) -> pd.DataFrame:
    return forecastWithAR(dataset, group_column, forecast_max_date)
# Using Pandas Function API
forecastDF = actualsDF.groupby(group_column)\
                  .applyInPandas(udf_uv_forecast,\
                                 schema=schema)
display (forecastDF)

NameError: name 'actualsDF' is not defined